# Baseline: Greedy Tree Packing

Implement a greedy approach based on the getting-started kernel:
1. Place trees one by one
2. Start far away at weighted random angle
3. Move toward center until collision
4. Back up until no collision
5. Apply rotation tightening (fix_direction) to minimize bounding box

In [1]:
import math
import random
from decimal import Decimal, getcontext
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.optimize import minimize_scalar
import time

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print('Libraries loaded')

Libraries loaded


In [2]:
# Build the index of the submission
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]
print(f'Total rows in submission: {len(index)}')

Total rows in submission: 20100


In [3]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

print('ChristmasTree class defined')

ChristmasTree class defined


In [4]:
def generate_weighted_angle():
    """Generates a random angle with a distribution weighted by abs(sin(2*angle))."""
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle

def initialize_trees(num_trees, existing_trees=None, num_attempts=10):
    """Greedy tree placement."""
    if num_trees == 0:
        return [], Decimal('0')

    if existing_trees is None:
        placed_trees = []
    else:
        placed_trees = list(existing_trees)

    num_to_add = num_trees - len(placed_trees)

    if num_to_add > 0:
        unplaced_trees = [
            ChristmasTree(angle=random.uniform(0, 360)) for _ in range(num_to_add)]
        if not placed_trees:
            placed_trees.append(unplaced_trees.pop(0))

        for tree_to_place in unplaced_trees:
            placed_polygons = [p.polygon for p in placed_trees]
            tree_index = STRtree(placed_polygons)

            best_px = None
            best_py = None
            min_radius = Decimal('Infinity')

            for _ in range(num_attempts):
                angle = generate_weighted_angle()
                vx = Decimal(str(math.cos(angle)))
                vy = Decimal(str(math.sin(angle)))

                radius = Decimal('20.0')
                step_in = Decimal('0.5')

                collision_found = False
                while radius >= 0:
                    px = radius * vx
                    py = radius * vy

                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor))

                    possible_indices = tree_index.query(candidate_poly)
                    if any((candidate_poly.intersects(placed_polygons[i]) and not
                            candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                        collision_found = True
                        break
                    radius -= step_in

                if collision_found:
                    step_out = Decimal('0.05')
                    while True:
                        radius += step_out
                        px = radius * vx
                        py = radius * vy

                        candidate_poly = affinity.translate(
                            tree_to_place.polygon,
                            xoff=float(px * scale_factor),
                            yoff=float(py * scale_factor))

                        possible_indices = tree_index.query(candidate_poly)
                        if not any((candidate_poly.intersects(placed_polygons[i]) and not
                                   candidate_poly.touches(placed_polygons[i]))
                                   for i in possible_indices):
                            break
                else:
                    radius = Decimal('0')
                    px = Decimal('0')
                    py = Decimal('0')

                if radius < min_radius:
                    min_radius = radius
                    best_px = px
                    best_py = py

            tree_to_place.center_x = best_px
            tree_to_place.center_y = best_py
            tree_to_place.polygon = affinity.translate(
                tree_to_place.polygon,
                xoff=float(tree_to_place.center_x * scale_factor),
                yoff=float(tree_to_place.center_y * scale_factor),
            )
            placed_trees.append(tree_to_place)

    all_polygons = [t.polygon for t in placed_trees]
    bounds = unary_union(all_polygons).bounds

    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor

    width = maxx - minx
    height = maxy - miny
    side_length = max(width, height)

    return placed_trees, side_length

print('Placement functions defined')

Placement functions defined


In [5]:
def get_all_vertices(trees):
    """Get all vertices from all trees."""
    vertices = []
    for tree in trees:
        coords = list(tree.polygon.exterior.coords)
        for x, y in coords:
            vertices.append((float(Decimal(x) / scale_factor), float(Decimal(y) / scale_factor)))
    return vertices

def compute_bbox_at_angle(vertices, angle_rad):
    """Compute bounding box side length after rotating all vertices by angle."""
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    
    rotated = [(x * cos_a - y * sin_a, x * sin_a + y * cos_a) for x, y in vertices]
    
    xs = [p[0] for p in rotated]
    ys = [p[1] for p in rotated]
    
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    
    return max(width, height)

def fix_direction(trees):
    """Optimize global rotation angle to minimize bounding box."""
    if len(trees) <= 1:
        return trees, None
    
    vertices = get_all_vertices(trees)
    
    def objective(angle_deg):
        return compute_bbox_at_angle(vertices, math.radians(angle_deg))
    
    # Search from 0 to 90 degrees (symmetry)
    result = minimize_scalar(objective, bounds=(0.001, 89.999), method='bounded')
    best_angle = result.x
    
    # Apply rotation to all trees
    cos_a = math.cos(math.radians(best_angle))
    sin_a = math.sin(math.radians(best_angle))
    
    new_trees = []
    for tree in trees:
        old_x = float(tree.center_x)
        old_y = float(tree.center_y)
        new_x = old_x * cos_a - old_y * sin_a
        new_y = old_x * sin_a + old_y * cos_a
        new_angle = float(tree.angle) + best_angle
        
        new_tree = ChristmasTree(
            center_x=str(new_x),
            center_y=str(new_y),
            angle=str(new_angle)
        )
        new_trees.append(new_tree)
    
    return new_trees, best_angle

print('fix_direction function defined')

fix_direction function defined


In [6]:
def compute_side_length(trees):
    """Compute the side length of the bounding square."""
    if not trees:
        return Decimal('0')
    
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    
    width = maxx - minx
    height = maxy - miny
    return max(width, height)

def compute_score(side_lengths):
    """Compute total score: sum(side^2 / n) for n=1 to 200."""
    total = 0
    for n, side in enumerate(side_lengths, 1):
        total += float(side) ** 2 / n
    return total

print('Scoring functions defined')

Scoring functions defined


In [7]:
# Run the greedy placement for all configurations
random.seed(42)

start_time = time.time()

tree_data = []
side_lengths = []
current_placed_trees = []

for n in range(1, 201):
    current_placed_trees, side = initialize_trees(n, existing_trees=current_placed_trees, num_attempts=10)
    
    # Apply fix_direction optimization
    if n > 1:
        optimized_trees, angle = fix_direction(current_placed_trees)
        new_side = compute_side_length(optimized_trees)
        if new_side < side:
            current_placed_trees = optimized_trees
            side = new_side
    
    side_lengths.append(side)
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f'n={n}: side={float(side):.6f}, elapsed={elapsed:.1f}s')
    
    # Store tree data for this configuration
    for tree in current_placed_trees:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

total_time = time.time() - start_time
print(f'\nTotal time: {total_time:.1f}s')

n=20: side=3.964713, elapsed=0.3s


n=40: side=5.560157, elapsed=0.8s


n=60: side=7.041550, elapsed=1.3s


n=80: side=7.976427, elapsed=2.0s


n=100: side=9.292022, elapsed=2.9s


n=120: side=9.824172, elapsed=3.9s


n=140: side=10.652730, elapsed=5.1s


n=160: side=11.425887, elapsed=6.4s


n=180: side=12.363815, elapsed=7.8s


n=200: side=12.779802, elapsed=9.5s

Total time: 9.5s


In [8]:
# Compute score
score = compute_score(side_lengths)
print(f'Total score: {score:.6f}')

Total score: 164.820038


In [9]:
# Create submission
cols = ['x', 'y', 'deg']
submission = pd.DataFrame(
    index=index, columns=cols, data=tree_data).rename_axis('id')

for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)
    
# Prepend 's' as required by format
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

print(submission.head(20))
print(f'\nShape: {submission.shape}')

                x           y          deg
id                                        
001_0        s0.0        s0.0  s230.193647
002_0        s0.0        s0.0  s236.959515
002_1  s-0.325998   s0.127379    s15.76974
003_0        s0.0        s0.0  s246.609375
003_1  s-0.342737    s0.07093   s25.419599
003_2   s0.042949   s0.498152  s300.216038
004_0        s0.0        s0.0  s330.151352
004_1   s-0.10903  s-0.332585  s108.961577
004_2   s-0.49016   s0.098706  s383.758016
004_3   s0.418223  s-0.357197  s220.145937
005_0        s0.0        s0.0  s341.792429
005_1  s-0.039678  s-0.347744  s120.602654
005_2  s-0.499995  s-0.002229  s395.399093
005_3   s0.481695  s-0.265461  s231.787014
005_4   s0.396135   s0.918192  s296.789648
006_0        s0.0        s0.0  s415.361244
006_1   s0.322318  s-0.136422  s194.171468
006_2  s-0.139292  s-0.480206  s468.967907
006_3   s0.390874   s0.386934  s305.355828
006_4  s-0.768641    s0.63968  s370.358463

Shape: (20100, 3)


In [10]:
# Save submission
submission.to_csv('/home/submission/submission.csv')
print('Submission saved to /home/submission/submission.csv')

Submission saved to /home/submission/submission.csv


In [11]:
# Validate submission format
import pandas as pd

sample = pd.read_csv('/home/data/sample_submission.csv', index_col='id')
ours = pd.read_csv('/home/submission/submission.csv', index_col='id')

print(f'Sample shape: {sample.shape}')
print(f'Our shape: {ours.shape}')
print(f'Index match: {(sample.index == ours.index).all()}')
print(f'Columns match: {list(sample.columns) == list(ours.columns)}')

Sample shape: (20100, 3)
Our shape: (20100, 3)
Index match: True
Columns match: True


In [13]:
# Let's evaluate the sample submission score
import pandas as pd

sample = pd.read_csv('/home/data/sample_submission.csv', index_col='id')

# Parse the sample submission
def parse_submission(df):
    """Parse submission and compute side lengths for each configuration."""
    side_lengths = []
    
    for n in range(1, 201):
        # Get trees for this configuration
        tree_ids = [f'{n:03d}_{t}' for t in range(n)]
        trees_df = df.loc[tree_ids]
        
        # Parse coordinates (remove 's' prefix)
        xs = trees_df['x'].str[1:].astype(float).values
        ys = trees_df['y'].str[1:].astype(float).values
        degs = trees_df['deg'].str[1:].astype(float).values
        
        # Create trees and compute bounding box
        trees = []
        for x, y, deg in zip(xs, ys, degs):
            tree = ChristmasTree(center_x=str(x), center_y=str(y), angle=str(deg))
            trees.append(tree)
        
        side = compute_side_length(trees)
        side_lengths.append(side)
        
        if n % 50 == 0:
            print(f'Parsed n={n}')
    
    return side_lengths

print('Parsing sample submission...')
sample_side_lengths = parse_submission(sample)
sample_score = compute_score(sample_side_lengths)
print(f'Sample submission score: {sample_score:.6f}')

Parsing sample submission...


Parsed n=50


Parsed n=100


Parsed n=150


Parsed n=200
Sample submission score: 173.652299
